In [5]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pygmt
import os
from scipy.spatial.distance import pdist
import time
from datetime import datetime
from multiprocessing import Pool, cpu_count
#  ========================================================================================================
def project_and_get_distances(points, reference_point):
    from pyproj import Transformer
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:32648", always_xy=True)
    
    # Project all points
    x, y = transformer.transform(points[:, 0], points[:, 1])
    x0, y0 = transformer.transform(reference_point[0], reference_point[1])
    xy = np.column_stack((x, y))
    ref_xy = np.array([x0, y0])
    distances = np.linalg.norm(xy - ref_xy, axis=1)
    return distances
def dist2ref(df,ref_point):
    from pyproj import Geod
    '''
    calculate the distance from the spicies coordinate to reference point
    Input is the data frame, the distance is a new column
    '''
    # Define geodetic system (WGS84)
    geod = Geod(ellps='WGS84')
    # Compute forward and back azimuths and distance
    # Function to compute distance
    def compute_distance(row):
        az12, az21, dist = geod.inv(ref_point[0], ref_point[1],row['Longitude'], row['Latitude'])
        az_rad = np.deg2rad(az12)
        dx = dist * np.sin(az_rad)  # East-west
        dy = dist * np.cos(az_rad)  # North-south
        return pd.Series([dist, dx, dy], index=['distance','d2x', 'd2y'])  # in meters
    # Apply row-wise
    df[['distance','d2x', 'd2y']] = df.apply(compute_distance, axis=1)
    return df  
    
    
def ripley_k(points, reference_point, radius, area):
    """
    Compute the Ripley-K function for a given reference point.
    + Input: 
    """
    # distances = np.linalg.norm(points - reference_point, axis=1)
    distances = project_and_get_distances(points, reference_point)
    count_within_radius = np.sum(distances <= radius) - 1  # exclude self
    K = (area / len(points)) * count_within_radius
    return K
def ripley_k_full(points, radius, area):
    from scipy.spatial.distance import pdist, squareform
    '''
    This calculate the ripley-K for full data set, not for single point
    '''
    n = len(points)
    dist_matrix = squareform(pdist(points))
    count = np.sum((dist_matrix <= radius)) - n  # subtract self-counts
    K = (area / (n * (n - 1))) * count
    return K
    
def derivative_ripley_k(points, reference_point, radius, area, delta_r=1e-5):
    """
    Compute the derivative of the Ripley-K function at a reference point with respect to the radius.
    """
    K_r = ripley_k(points, reference_point, radius, area)
    K_r_plus_delta = ripley_k(points, reference_point, radius + delta_r, area)
    dK_dr = ((K_r_plus_delta - K_r) / delta_r)
    # print("> Radius:",radius)
    if radius == 0:
        gr = 0
    else:
        gr = dK_dr/(2*np.pi*radius)
    return gr

def derivative_ripley_k_range(points, reference_point, max_radius, area, num_steps, delta_r):
    """
    Compute the derivative of the Ripley-K function over a range of radii from 0 to max_radius.

    Parameters:
    points (numpy.ndarray): An Nx2 array of point coordinates.
    reference_point (numpy.ndarray): A 1x2 array representing the reference point coordinates.
    max_radius (float): The maximum radius for the range.
    area (float): The area of the study region (needed for normalization).
    num_steps (int): The number of steps (or radii values) in the range.
    delta_r (float): A small change in the radius to compute the finite difference.

    Returns:
    radii (numpy.ndarray): An array of radii.
    dK_dr_values (numpy.ndarray): The corresponding derivative values of the Ripley-K function at each radius.
    """
    # Generate an array of radii from 0 to max_radius
    radii = np.linspace(0, max_radius, num_steps)
    # dK_dr_values = np.zeros_like(radii)
    gr = np.zeros_like(radii)

    # Compute the derivative of the Ripley-K function at each radius
    for i, r in enumerate(radii):
        # dK_dr_values[i] = derivative_ripley_k(points, reference_point, r, area, delta_r)
        gr[i] = derivative_ripley_k(points, reference_point, r, area, delta_r)
        
    return radii, gr
# Helper function to apply to each point
def compute_gr_for_point(args):
    '''
    Parallel computing for this GR
    '''
    point, radius, area, step = args
    radii, dk_dr_value = derivative_ripley_k_range(points_global, point, radius, area, step, 1)
    n = len(radii)
    return pd.DataFrame({
        'Longitude': np.full(n, point[0]),
        'Latitude': np.full(n, point[1]),
        'radius_m': radii,
        'gr': dk_dr_value
    })
    
#  ========================================================================================================
if __name__ == "__main__":
    # Start timing
    t0 = time.time()
    start_time_str = datetime.now().strftime("Start time: %a %b %d %H:%M:%S %Y")
    print(start_time_str)
    # ----------------------------------------------------------------------------
    pwd = os.getcwd()
    #
    infile = os.path.join(pwd,"output","01_all_IVI_sorted.txt")
    #
    all_data_file = os.path.join(pwd,"output","01_all_distances.txt")
    # read the data and kip 2 rows due to format error
    data_ini = pd.read_csv(infile, sep=',',skiprows=0,header=0,)
    all_data = pd.read_csv(all_data_file, sep=',',skiprows=0,header=0,)
    # data_ini.head(20)
    # Select the data that less than 75% of the cumulative IVI
    data_selected = data_ini[data_ini['IVIcu'] <= 75.]; data_selected.reset_index(drop=True,inplace=True)
    # Now filter the data with selected spices 
    all_data_selected = all_data.head(0)
    for i,sourcecodenow in enumerate(data_selected['sourcecode']):
        all_data_now = all_data[all_data['source_code']==sourcecodenow];
        all_data_now.reset_index(drop=True,inplace=True)
        
        all_data_selected = pd.concat([all_data_selected,all_data_now],axis=0)
    all_data_selected.reset_index(drop=True,inplace=True)
    points = all_data_selected[['source_long', 'source_lat']].drop_duplicates().to_numpy()
    # # Area of study area
    # area = area_deg_plain(data['Sourcelong'].min(),data['Sourcelat'].min(),data['Sourcelong'].max(),data['Sourcelat'].max())
    # # radius
    # radius=avg_dist/111320; # in deg
    # #
    dk_dr_values=[];
    radiis=[]
    ref_points_long=[];
    ref_points_lat=[];
    # calculate the average dist
    all_data_selected['pair'] = all_data_selected.apply(lambda row: tuple(sorted([row['source_long'], row['source_lat']])),axis=1)
    all_data_selected_unique = all_data_selected.drop_duplicates(subset='pair')
    avg_dist = all_data_selected_unique['Dist'].mean()
    # radius
    # radius=avg_dist/111320; # in deg
    radius = int(avg_dist);
    area = data_ini['Area'].unique()[0]
    # area = data_ini['Area'].unique()[0]/10000 # convert to hecta
    # Now calculate the Gr for all the data points
    points_global = all_data_selected[['source_long', 'source_lat']].drop_duplicates().to_numpy()
    num_cpus = min(40, cpu_count()) # 40 Cpu or max
    print("Calculating GR with critical radius = {} (m), area = {} (m) using {} CPUs...".format(radius,area,num_cpus))
    # Prepare argument list for parallel pool
    args_list = [(tuple(point), radius, area, int(avg_dist)+1) for point in points_global]
    # Parallel computation
    with Pool(processes=num_cpus) as pool:
        results = pool.map(compute_gr_for_point, args_list)

    dataout = pd.concat(results, ignore_index=True)
    
    # for i,reference_point in enumerate(points):
    #     # k_value = ripley_k(points, reference_point, radius, area)
    #     # radii, dk_dr_value = derivative_ripley_k_range(points, reference_point, radius, area, int(avg_dist)+1, all_data_selected_unique['distance'].min())
    #     radii, dk_dr_value = derivative_ripley_k_range(points, reference_point, radius, area, int(avg_dist)+1, 1)
    #     # print(len(radii)
    #     ref_points_long = np.append(ref_points_long,np.full(len(radii),reference_point[0]))
    #     ref_points_lat = np.append(ref_points_lat,np.full(len(radii),reference_point[1]))
    #     dk_dr_values = np.append(dk_dr_values,dk_dr_value)
    #     # radiis = np.append(radiis,radii*111320)
    #     radiis = np.append(radiis,radii)
        
    #     data_out = np.vstack((ref_points_long.flatten(),ref_points_lat.flatten(),radiis.flatten(),dk_dr_values.flatten()))
    # dataout = pd.DataFrame(data_out.T,columns=['Longitude','Latitude','radius_m','gr'])
    ref_point=[108.5754,14.4991]
    dataout = dist2ref(dataout, ref_point)
    # ====================================================================================================================
    dataout.to_csv(os.path.join(pwd,"output","02_species_GR.txt"),index=False,sep='\t',encoding='utf-8')
    print(' >>>> Export the output file: {}'.format(os.path.join(pwd,"output","02_species_GR.txt")))
    dataout['radius_m'] = dataout['radius_m'].apply(lambda x: float(f"{x:.1f}"))
    # End timing
    t1 = time.time()
    end_time_str = datetime.now().strftime("End time: %a %b %d %H:%M:%S %Y")  
    total_time = t1 - t0
    print(end_time_str)
    print("Total process time: {:.2f} seconds".format(total_time))
    print("done!")

Start time: Mon Jul 28 11:51:04 2025
Calculating GR with critical radius = 56 (m), area = 13386.634249231713 (m) using 40 CPUs...
 >>>> Export the output file: /data/longhv/Projects/VietNga_species_distribution_example/spatial_mapping/output/02_species_GR.txt
End time: Mon Jul 28 11:51:20 2025
Total process time: 16.25 seconds
done!


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pygmt
import os
from scipy.spatial.distance import pdist
# ======================================================================
'''
Plot the figure for each radius distances
'''
# ======================================================================
pwd = os.getcwd();
#
indata = pd.read_csv(os.path.join(pwd,"output","02_species_GR.txt"),sep="\s+",header=0)
# Output figure
out_fig_dir = os.path.join(pwd,"figures","GR_radius")
os.makedirs(out_fig_dir,exist_ok=True)
# ======================================================================
for i,radius in enumerate(indata['radius_m']):
    # Extract the data
    data_now = indata[indata['radius_m']==radius]
    data_now.reset_index(drop=True,inplace=True)
    # Now plot the figure
    fig = pygmt.Figure()
    pygmt.config(FONT_LABEL="14p,Times-Bold,black",
                 FONT_TITLE="15p,Times-Bold,black",
                 FONT_ANNOT_PRIMARY="10p,Times-Bold,black",
                 FONT_ANNOT_SECONDARY="10p,Times-Bold,black"
                )
    area = [0,np.ceil(int(indata['d2x'].max())/10)*10,0,np.ceil(int(indata['d2y'].max())/10)*10]
    # grid = pygmt.datasets.load_earth_relief(resolution="01s", region=area)
    # fig.grdimage(grid=grid, projection="X5i/5i", frame="a", cmap="geo")
    # fig.colorbar(frame=["a100", "x+lElevation", "y+lm"])
    fig.basemap(region=area, projection="X5i/5i", frame=['xafg+l"Easting(m)"','yafg+l"Northing(m)"', 'WSne+t"G(r) map with radius of {} (m)"'.format(radius)])
    gr_grd = pygmt.surface(x=data_now['d2x'], y=data_now['d2y'], z=data_now['gr'],
                            region=area, spacing='0.1/0.1') # 1 metter per 1 metter
    # pygmt.makecpt(cmap=["0   blue","1   white","{}   red".format(np.ceil((data_now['gr'].max())*10)/10)],series=[0, (np.ceil((data_now['gr'].max())*10)/10), 0.01])
    with open("custom.cpt", "w") as f:
        f.write(
            """
            0	blue	1	white
            1	white	{}	red
            B	blue
            F	white
            N	gray
            """.format(int(data_now['gr'].max()))
            )
    # -------------------------------------------------------------------
    # pygmt.makecpt(cmap=cmap,series=[0, 7, 0.01])
    fig.grdimage(grid=gr_grd,
                 # shading=True,
                 # projection="X5i/5i",
                 cmap="custom.cpt")
    fig.colorbar(
        cmap="custom.cpt",
        position="JMR+o0.25i/0.0c+w5.0i/0.25c+n",
        box=None,
        frame=["xaf+lG(r)"],
        # scale=1,  # Adjust the scale
        # B="x10"
    )
    fig.savefig(os.path.join(out_fig_dir,"00_distribution_gr_{}_km.png".format(radius)),crop=True, dpi=300, transparent=False)
    # fig.show()
print("Done!")

surface [WARNING]: 1 unusable points were supplied; these will be ignored.
surface [WARNING]: You should have pre-processed the data with block-mean, -median, or -mode.
surface [WARNING]: Check that previous processing steps write results with enough decimals.
surface [WARNING]: Possibly some data were half-way between nodes and subject to IEEE 754 rounding.
surface [WARNING]: Input data lie exactly on a plane.
grdimage [ERROR]: Z-slice around line 1 with dz <= 0
[Session pygmt-session (10)]: Error returned from GMT API: GMT_CPT_READ_ERROR (8)
[Session pygmt-session (10)]: Error returned from GMT API: GMT_CPT_READ_ERROR (8)
grdimage [ERROR]: Unable to save current CPT file to /home/longhv/.gmt/sessions/gmt_session.82162/gmt.1.cpt !
grdimage [ERROR]: Failed to read CPT custom.cpt.
colorbar [ERROR]: Z-slice around line 1 with dz <= 0
[Session pygmt-session (12)]: Error returned from GMT API: GMT_CPT_READ_ERROR (8)
[Session pygmt-session (12)]: Error returned from GMT API: GMT_CPT_READ_ER